<img src="NotebookAddons/blackboard-banner.jpg" width="100%" />
<font face="Calibri">
<br>
<font size="5"> <b>Exploring SAR Time Series Data for Flood Monitoring</b></font>

<br>
<font size="4"> <b> Franz J Meyer; University of Alaska Fairbanks </b> <br>
<img style="padding:7px;" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right" /></font>

<font size="3">This notebook introduces you to the time series signatures associated with flooding. The data analysis is done in the framework of *Jupyter Notebooks*. The Jupyter Notebook environment is easy to launch in any web browser for interactive data exploration with provided or new training data. Notebooks are comprised of text written in a combination of executable python code and markdown formatting including latex style mathematical equations. Another advantage of Jupyter Notebooks is that they can easily be expanded, changed, and shared with new data sets or newly available time series steps. Therefore, they provide an excellent basis for collaborative and repeatable data analysis. <br>

<b>This notebook covers the following data analysis concepts:</b>

- How to load time series stacks into Jupyter Notebooks and how to explore image content using basic functions such as mean value calculation and histogram analysis.
- How to extract time series information for individual pixels of an image.
- Typical time series signatures over flood affected areas.
</font>


</font>

<hr>
<font face="Calibri" size="5" color='rgba(200,0,0,0.2)'> <b>Important Notes about Binder</b> </font>
<br><br>
<font face="Calibri" size="3"> <b>The Binder server will automatically shutdown when left idle for more than 10 minutes. Your notebook edits will be lost when this happens. You will need to relaunch the binder to continue working in a fresh copy of the notebook.</b></font>
    <br><br>
    <font face="Calibri" size="4"><b>How to Save your Notebook Edits</b></font>
        <br><br>
<font face="Calibri" size="3"><b>The Easy Way</b>
    <br>
Click on the Jupyter logo at the top left of the screen to access the file manager. Download the notebook, then upload and run it the next time you restart the server.
    <br><br>
<b>The Better, More Complicated Way</b>
    <br>
This solution requires some knowledge of git. Fork the <a href="https://github.com/asfadmin/asf-jupyter-notebooks" target="_blank">asf-jupyter-notebook repository</a> and update the url for the Binder launch button to the url of your fork. The url to edit can be found in the first line of the README.md file for this branch. Once you have your own fork, push any notebook changes to it prior to shutting down the server or allowing it to time out.  </font>
<br><br>
</font>
<hr>

# Importing Relevant Python Packages

<font face="Calibri">
<font size="3">In this notebook we will use the following scientific libraries:

<ol type="1">
    <li> <b><a href="https://www.gdal.org/" target="_blank">GDAL</a></b> is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.</li>
    <li> <b><a href="http://www.numpy.org/" target="_blank">NumPy</a></b> is one of the principal packages for scientific applications of Python. It is intended for processing large multidimensional arrays and matrices, and an extensive collection of high-level mathematical functions and implemented methods makes it possible to perform various operations with these objects. </li>
    <li> <b><a href="https://matplotlib.org/index.html" target="_blank">Matplotlib</a></b> is a low-level library for creating two-dimensional diagrams and graphs. With its help, you can build diverse charts, from histograms and scatterplots to non-Cartesian coordinates graphs. Moreover, many popular plotting libraries are designed to work in conjunction with matplotlib. </li>
</font>

In [ ]:
import os # for chdir, getcwd, path.basename, path.exists
from math import ceil
from glob import glob

from osgeo import gdal # for GetRasterBand, Open, ReadAsArray
import numpy as np #for log10, mean, percentile, power
import pyproj # for Proj, transform

import matplotlib.pyplot as plt # for add_subplot, axis, figure, imshow, legend, plot, set_axis_off, set_data,
                                # set_title, set_xlabel, set_ylabel, set_ylim, subplots, title, twinx
import matplotlib.patches as patches  # for Rectangle
import matplotlib.animation as an # for FuncAnimation

from matplotlib import rc 
from IPython.display import HTML
plt.rcParams.update({'font.size': 12})

<font face="Calibri" size="3"><b>Setup matplotlib plotting inside the notebook:</b></font>

In [ ]:
%matplotlib notebook

<hr>

# Load Data Stack

<font face="Calibri">
<img src="https://www.elcomercio.com/files/article_main/uploads/2017/03/29/58dc6b8cd9665.jpeg" style="float: right; width: 50%; margin-left: 3%; margin-bottom: 0.5em;" /><font size="3"> This notebook allows for the analysis of a recent flooding events north of Guayaquil, Ecuador. The analysis is done using a 17-image deep Sentinel-1 data stack (VV only). 

Ecuador and other countries in western South America experienced widespread <b>flooding</b> during the 2016-2017 winter (see picture by Bolívar Velasco/EL COMERCIO). <a href='https://www.eluniverso.com/noticias/2017/03/01/nota/6068059/arboles-se-caen-medio-torrencial-lluvia-ayer'>Guayaquil in Guayas</a> was among the affected regions, as precipitation in March 2017 was well above average. The increased precipitation was associated with a Coastal Niño event. The data provided allows to study the extent and progression of the flooding in the year 2016-2017 just north of Guayaquil.
</font></font>

<font face="Calibri" size="3">Before we get started, let's first <b>create a working directory for this analysis and change into it:</b> </font>

In [ ]:
name="flood"
path = f"/home/jovyan/{name}"
if not os.path.exists(path):
    os.mkdir(path)
os.chdir(path)
print(f"Current working directory: {os.getcwd()}")

<font face="Calibri" size="3">We will <b>retrieve the relevant data</b> from an <a href="https://aws.amazon.com/" target="_blank">Amazon Web Service (AWS)</a> cloud storage bucket <b>using the following command</b>:</font></font>

In [ ]:
time_series_path = f"s3://asf-jupyter-data/{name}.tar.gz"
time_series = os.path.basename(time_series_path)
!aws --no-sign-request --region us-east-1 s3 cp $time_series_path $time_series

<font face="Calibri" size="3"> Now, let's <b>unzip the file (overwriting previous extractions) and clean up after ourselves:</b> </font>

In [ ]:
!tar -xvzf {name}.tar.gz

# Define Data Directory and Path to VRT

<font face="Calibri" size="3"><b>Create a variable containing the VRT filename and the image acquisition dates:</b></font>

In [ ]:
def get_dates(path):
    pths = glob(path)
    pths.sort()
    dates = []
    for pth in pths:
        date = os.path.basename(pth).split('T')[1].split('_')[1]
        date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
        dates.append(date)
    return dates

def get_dates_sub(path):
    pths = glob(path)
    pths.sort()
    dates = []
    for pth in pths:
        date = os.path.basename(pth).split('_')[0]
        date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
        dates.append(date)
    return dates

In [ ]:
tiff_paths = f"{path}/tiffsflood/*.tif*"
dates = get_dates(tiff_paths)
print(dates)

In [ ]:
polarization = 'VV'
image_file = f"stack{name}_{polarization}.vrt"

<hr>

# Data Exploration with an Animation

<font face="Calibri" size="3"><b>Read the data</b></font>

In [ ]:
print(image_file)
img = gdal.Open(image_file)
band = img.GetRasterBand(1)
raster0 = band.ReadAsArray()
band_number = 0 # Needed for updates
rasterstack = img.ReadAsArray()


<font face="Calibri" size="3">Before analyzing the data, decide whether to use <b>linear or logarithmic scaling</b></font>

In [ ]:
use_dB = True

def convert(raster, use_dB=use_dB):
    # some Python trickery: 
    # if you call the convert function later, you can set the keyword 
    # argument use_dB to True or False
    # if you do not provide a keyword argument, the value that you set
    # above (when defining the function) is used
    if use_dB:
        return 10 * np.log10(raster)
    else:
        return raster


<font face="Calibri" size="3"> Let's create an <b>animation</b> to get an idea of where and when flooding might have occurred.</font>

In [ ]:
%%capture 
figani = plt.figure(figsize=(12, 7))
axani = figani.subplots()
axani.axis('off')

rasterstack_ = convert(rasterstack)

imani = axani.imshow(rasterstack_[0,...], cmap='gray', vmin=np.nanpercentile(rasterstack_, 1), 
               vmax=np.nanpercentile(rasterstack_, 99))
axani.set_title("{}".format(dates)) #time_index[0].date()))

def animate(i):
    axani.set_title(dates[i])
    imani.set_data(rasterstack_[i,...])

# Interval is given in milliseconds
ani = an.FuncAnimation(figani, animate, frames=rasterstack_.shape[0], interval=300)
rc('animation', embed_limit=40971520.0)  # We need to increase the limit maybe to show the entire animation

<font face="Calibri" size="3"><b>Render</b></font>

In [ ]:
HTML(ani.to_jshtml())

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u>:  </font> Backscatter dynamics</b> </font>

<font face="Calibri" size="3"> What is the most striking change in February 2017? Can you explain why the backscatter changes the way it does during that period?
</font>
</div>
<hr>

# Create Minimum Image to Identify Inundated Areas

<font face="Calibri" size="3"> As flooding is often associated with very low backscater, we first compute the minimum backscatter for each pixel to get a first impression of areas that could have been flooded during the entire period. </font>


<font face="Calibri" size="3"> The following line <b>calculates the minimum backscatter per pixel</b> across the time series: </font>

In [ ]:
rasterstack_masked = np.ma.masked_where(rasterstack==0, rasterstack)
temporal_min = np.nanmin(convert(rasterstack_masked), axis=0)

<font face="Calibri" size="3"> <b>Write a class to create an interactive plot from which we can select interesting image locations for a time series.</b></font>

In [ ]:
class pixelPicker:
    def __init__(self, image, width, height):
        self.x = None
        self.y = None
        self.fig = plt.figure(figsize=(width, height))
        self.ax = self.fig.add_subplot(111, visible=False)
        self.rect = patches.Rectangle(
            (0.0, 0.0), width, height, 
            fill=False, clip_on=False, visible=False
        )
       
        self.rect_patch = self.ax.add_patch(self.rect)
        self.cid = self.rect_patch.figure.canvas.mpl_connect('button_press_event', 
                                                             self)
        self.image = image
        self.plot = self.gray_plot(self.image, fig=self.fig, return_ax=True)
        self.plot.set_title('Select a Point of Interest')
        
        
    def gray_plot(self, image, vmin=None, vmax=None, fig=None, return_ax=False):
        '''
        Plots an image in grayscale.
        Parameters:
        - image: 2D array of raster values
        - vmin: Minimum value for colormap
        - vmax: Maximum value for colormap
        - return_ax: Option to return plot axis
        '''
        if vmin is None:
            vmin = np.nanpercentile(self.image, 1)
        if vmax is None:
            vmax = np.nanpercentile(self.image, 99)
        ax = fig.add_axes([0.1,0.1,0.8,0.8])
        ax.imshow(image, cmap=plt.cm.gist_gray, vmin=vmin, vmax=vmax)
        if return_ax:
            return(ax)
        
    
    def __call__(self, event):
        print('click', event)
        self.x = event.xdata
        self.y = event.ydata
        for pnt in self.plot.get_lines():
            pnt.remove()
        plt.plot(self.x, self.y, 'ro')

<font face="Calibri" size="3"> Now we are ready to plot the minimum image. <b>Click a point interest for which you want to analyze radar brightness over time</b>: </font>

In [ ]:
fig_xsize = 7.5
fig_ysize = 7.5
my_plot = pixelPicker(temporal_min, fig_xsize, fig_ysize)

<font face="Calibri" size="3"><b>Save the selected coordinates</b>: </font>

In [ ]:
try:
    sarloc = (ceil(my_plot.x), ceil(my_plot.y))
    print(sarloc)
except TypeError:
    print("You must select a a Point of Interest from the plot in the previous cell.")

# Plot SAR Brightness Time Series at Point Locations

<font face="Calibri" size="3"> We will pick a pixel location identified in the SAR image above and plot the time series for this identified point. By focusing on image locations undergoing deforestation, we should see the changes in the radar cross section related to the deforestation event.
    
First, for processing of the imagery in this notebook we generate a list of image handles and retrieve projection and georeferencing information. We also define a function for mapping image pixels to a geographic projection</font> 

In [ ]:
img_handle = gdal.Open(image_file)
geotrans = img_handle.GetGeoTransform()
proj = img_handle.GetProjection()
xsize = img_handle.RasterXSize
ysize = img_handle.RasterYSize
bands = img_handle.RasterCount
projlatlon = pyproj.Proj('EPSG:4326') # WGS84
projstring = proj.split('[')[-1][:-2].split(',')[-1][1:-1]
projimg = pyproj.Proj(f'EPSG:{projstring}')

def geolocation(x, y=None, latlon=True):
    if len(x) == 2:
        y = x[1]
        x = x[0]
    ref_x=geotrans[0]+sarloc[0]*geotrans[1]
    ref_y=geotrans[3]+sarloc[1]*geotrans[5]
    if latlon:
        proj = pyproj.Transformer.from_crs(int(projstring), 4326, always_xy=True)
        ref_y, ref_x = proj.transform(ref_x, ref_y)
        #ref_y, ref_x = pyproj.transform(projimg, projlatlon, ref_x, ref_y)
    return (ref_x, ref_y)

<font face="Calibri" size="3"> Now, let's <b>pick a rectangle around a center pixel defined in variable <i>sarloc</i></b>...</font>

In [ ]:
extent = (5, 5) # choose a 5 by 5 rectangle
latlon = True # if False: return utm coordinates

refsarloc = geolocation(sarloc, latlon=latlon)
projsymbol = '°' if latlon else 'm'

<font face="Calibri" size="3">... and <b>extract the time series</b> for this small area around the selected center pixel in a memory-efficient way (needed for larger stacks):</font> 

In [ ]:
plt.rcParams.update({'font.size': 9})
bs_aggregated = []
for band in range(bands):
    rs = img_handle.GetRasterBand(band+1).ReadAsArray(sarloc[0], sarloc[1], extent[0], extent[1])
    rs_mean = convert(np.nanmean(rs))
    bs_aggregated.append(rs_mean)

fig, ax = plt.subplots(1, 1, figsize=(9, 5))
labeldB = 'dB' if use_dB else 'linear'
ax.plot(dates, bs_aggregated, color='k', marker='o', markersize=3)
plt.ylim((-28, 6))
ax.set_xlabel('Date')
plt.xticks(rotation=90)
plt.gcf().subplots_adjust(bottom=0.25)
ax.set_ylabel(f'Sentinel-1 $\gamma^0$ [{labeldB}]')

plt.grid()
_ = fig.suptitle(f'Location: {refsarloc[0]:.3f}{projsymbol} {refsarloc[1]:.3f}{projsymbol}')
# fig.tight_layout() 
figname = "RCSTimeSeries-" + f'{refsarloc[0]:.3f}{projsymbol} {refsarloc[1]:.3f}{projsymbol}' + '.png'
plt.savefig(figname, dpi=300, transparent='true')

<br>
<div class="alert alert-success">
<font face="Calibri" size="5"> <b> <font color='rgba(200,0,0,0.2)'> <u>EXERCISE</u>:  </font> Explore Time Series at Different Point Locations </b> </font>

<font face="Calibri" size="3"> Go back to the above map, pick a different point and plot its time series by running through the code cells after the map again. Can you interpret and attribute the changes at various locations? Apart from the flooding, what other patterns do you observe? Pick several points and explore the data set.
</font>
</div>
<br>
<hr>

# Version Log

<font face="Calibri" size="2"> <i>SAR Training Materials - Version 1.3.1 - September 2020</i>
    <br>
        <i>Version Changes:</i>
    <br>
    <i>- remove unecessary reliances on asf_notebook, use os functions</i>
        <br>
    <i>- add "--no-sign-request --region us-east-1" to aws s3 cp from public bucket</i>
    
</font>